# Smart Beta Portfolio

The Smart Beta strategy in finance is an investment approach that combines elements of both passive and active investing. Unlike traditional passive strategies that track market-capitalization-weighted indexes (like the S&P 500), Smart Beta strategies aim to enhance returns, reduce risk, or improve diversification by weighting assets based on alternative factors or metrics. These factors might include value, size, momentum, volatility, or dividends.

Smart Beta strategies systematically follow rules-based approaches to select and weight assets, and they typically offer a middle ground between purely passive index investing and active stock picking. The goal is to capture specific risk premiums associated with the chosen factors, potentially leading to better risk-adjusted returns compared to traditional market-cap-weighted portfolios.


In [48]:
import pandas as pd
import numpy as np
import sys
import os
# Add the parent directory to the sys.path
sys.path.append(os.path.join(os.path.dirname('Practice'), '..'))

from Practice import project_test3, project3_helper

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import plotly.graph_objects as go

import plotly.offline as offline_py
offline_py.init_notebook_mode(connected=True)


In [61]:
df = pd.read_csv('https://raw.githubusercontent.com/rbpal/01-qt-project-03-smart-beta-portfolio-optimization/main/eod-quotemedia.csv')

# get tickers of top 20% dollar traded 
percent_top_dollar = 0.2

# group the data by ticker and sum the dollar traded
dollar_traded = df.groupby('ticker').apply(lambda row: sum(row['adj_volume'] * row['adj_close']),include_groups=False)

# sort by ascending and get the top x percent
high_volume_symbols = dollar_traded.sort_values(ascending=True).head(int(len(dollar_traded) * percent_top_dollar)).index.values.tolist()

# filter the dataframe to only include the top x percent
df = df[df['ticker'].isin(high_volume_symbols)]

# get close, volume and dividend data
close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')
volume = df.reset_index().pivot(index='date', columns='ticker', values='adj_volume')
dividends = df.reset_index().pivot(index='date', columns='ticker', values='dividends')


## Part 1: Smart Beta Portfolio

In Part 1 of this project, we'll build a portfolio using dividend yield to choose the portfolio weights. A portfolio such as this could be incorporated into a smart beta ETF. You'll compare this portfolio to a market cap weighted index to see how well it performs.

For this exercise we will simulate a market cap weighted index. 

### Index weights

The index we'll be using is based on large dollar volume stocks. 

In [89]:
volume.shape

(1009, 99)

In [108]:
def generate_dollar_volume_weights(close, volume):
    """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
    """
    assert close.index.equals(volume.index)
    assert close.columns.equals(volume.columns)
    
    total_dollar_date = (close * volume).sum(axis=1)
    # calculate dollar volume by multiplying close and volume for each ticker and divide by total dollar volume traded that date
    dollar_volume = (close * volume).T/total_dollar_date
    
    
    return dollar_volume.T

index_weights = generate_dollar_volume_weights(close, volume)

In [110]:
index_weights

ticker,AEE,AES,AIV,AIZ,AJG,ALGN,ALLE,AME,ANSS,AOS,...,VNO,VRSK,VRSN,WAT,WLTW,WRK,XL,XRAY,XRX,XYL
date,,,,,,,,,,,,,,,,,,,,,
2013-07-01,0.01259361,0.01460270,0.00623275,0.01042729,0.00404777,0.00724774,NaN,0.01639601,0.00975799,0.00801810,...,0.01433121,0.01369001,0.01608420,0.02919710,NaN,NaN,0.01487717,0.00740137,0.00854346,0.01030244
2013-07-02,0.01094040,0.01436903,0.00891847,0.00498565,0.00275748,0.00472873,NaN,0.01771242,0.00645371,0.00529273,...,0.01217404,0.00736483,0.01530024,0.01343136,NaN,NaN,0.01495030,0.00852222,0.00817979,0.00874872
2013-07-03,0.00817890,0.01386479,0.01121132,0.00672836,0.00280110,0.00596251,NaN,0.01532232,0.00889533,0.00781099,...,0.02344512,0.00808804,0.01363098,0.00668016,NaN,NaN,0.01319838,0.01549573,0.01272771,0.00406890
2013-07-05,0.01067245,0.01495360,0.01100171,0.00865054,0.00571854,0.00553610,NaN,0.01499963,0.00769681,0.00448978,...,0.02747365,0.01407238,0.01048479,0.01176924,NaN,NaN,0.01788035,0.00929962,0.01441515,0.00593330
2013-07-08,0.01121470,0.01496035,0.01095179,0.00850345,0.00402173,0.00705648,NaN,0.01188778,0.00751759,0.00845637,...,0.02564589,0.01267450,0.01676205,0.01173339,NaN,NaN,0.01246654,0.00610847,0.01074408,0.00608896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-06-26,0.01254198,0.00955300,0.00607569,0.00882431,0.01044192,0.01809762,0.00314579,0.01146623,0.00821107,0.00471589,...,0.00858149,0.01065831,0.00804617,0.00997248,0.01945874,0.01213659,0.00726603,0.00890074,0.02472632,0.01780853
2017-06-27,0.01790755,0.00821381,0.01185240,0.00566173,0.01040056,0.02100870,0.00580212,0.00735270,0.01087908,0.00533250,...,0.00871666,0.01191756,0.00599618,0.00875189,0.01738626,0.01042592,0.01139829,0.01833571,0.01756196,0.01060111
2017-06-28,0.00884463,0.01078475,0.01086339,0.00590934,0.00782771,0.01688374,0.00324515,0.00778010,0.01072091,0.00469796,...,0.00841806,0.00948264,0.00680526,0.01271299,0.01904776,0.01179792,0.00753900,0.01167942,0.01426733,0.01093590
